In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

tf.__version__

'2.9.0'

从创建数据集开始，逐步研究如何构建Char-RNN

并从Anderej Karpathy的CharRNN项目中下载数据

In [2]:
# 使用keras.get_file()来下载莎士比亚的所有作品
# shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = "D:/temp_files/datasets/shakespeare/shakespeare.txt"
with open(filepath) as f:
    shakespeare_text = f.read()

print(shakespeare_text[:20])

First Citizen:
Befor


必须将每个字符编码为整数

    使用keras的Tokenizer类，为文本添加一个分词器，它会找到文本中使用的所有字符，并将它们映射到不同的字符ID，从1开始（不从0，所以可用0进行屏蔽）

In [3]:
# 看看有哪些字符
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

创建分词器

---

In [3]:
# 使用tokenizer类分词,char_level=True来得到字符集编码，而不是单词集编码
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
# 请注意，默认情况下，该分词器将文本转换为小写（def:lower=True）

---

In [5]:
# text->sequences
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [6]:
# sequences->text
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

---

In [4]:
# number of distinct characters
max_id = len(tokenizer.word_index)
# total number of characters
dataset_size = tokenizer.document_count

In [5]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

<font color = 'red'>简而言之，将时间序列划分集合并不是一件容易的事，可能要按时间划分，也可能是在所有时间上分层抽样，如何划分是一门学问，要具体情况具体分析</font>

---


## 将顺序数据集切成多个窗口

<font color = "yellow">
训练集现在由一个超过一百万字符的单个序列组成，如果直接输入并训练，就相当于产生了一个超过一百万层的深层网络，时间序列非常长
</font>

---
<font color = "green">

因此，使用window()将这个百万序列划分成许多小窗口

</font>

---

* 时间截断反向传播

    数据集中的每个实例将是整个文本的很短的子字符串，并且RNN仅仅在这些子字符串的长度上展开

### 基础划分举例

    在开始之前，首先看一看如何把一个序列拆分成多个批次的随机洗牌窗口

    把0~14，分成多个长度为5的窗口，每个窗口都左移两个单位，再对它们进行洗牌，最后把他们分成输入（除了尾值）和标签（除了头值）

In [9]:
# n_steps = 5
# dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
# dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
# dataset = dataset.flat_map(lambda window: window.batch(n_steps))
# dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
# dataset = dataset.batch(3).prefetch(1)
# for index, (X_batch, Y_batch) in enumerate(dataset):
#     print(f"Batch{index}: ")
#     print(f"X_Batch:\n{X_batch.numpy()}")
#     print(f"Y_Batch:\n{Y_batch.numpy()}")

### 在数据集上切分

    使用window()方法

window()方法会创建不重叠的窗口，但如果使用shift=1,则窗口分布则会变成，0~100/1~101,

如果使用drop_remainder=True,则丢弃了那些长度小于101的窗口，从而保证所有窗口长度一致

In [7]:
n_steps = 100
# target = input shifted 1 character ahead
window_length = n_steps + 1 
# 创建一个包含窗口的数据集，每个窗口也表示为一个数据集，列表的列表
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
# batch()方法将嵌套的长度不一的数据集，分为嵌套的长度均匀的数据集
# 由于每个窗口长度相同，因此每个窗口都获得一个张量
# flat_map()将嵌套的数据集转换为展平的数据集
dataset = dataset.flat_map(lambda window: window.batch(window_length))

现在得到了一个展平的数据集（张量），由于当训练集中的实例独立且同分布相同时，梯度下降效果最好，因此需要对这些窗口进行混洗

然后，可以批处理(map)这些窗口并将输入和目标分开

In [11]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

一般应将类别输入特征，独热编码或者嵌入

此处使用独热编码

In [12]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


## 创建和训练Char-RNN模型
要基于前100个字符来预测下一个字符，我们可以使用两个有GRU层的RNN，每个GRU层有128个单元，输入和隐藏状态的dropout率均为20%

输出层是一个时间分布的Dense层，这一次该层必须有max_id个单元，因为max_id表示文本中不同的字符数

每个时间不长的输出概率总和为1，因此在输出层使用softmax

训练时间可能非常久

In [13]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')
cb =  keras.callbacks.ModelCheckpoint("charRNN_cb.h5")
history = model.fit(dataset, epochs=10, callbacks=[cb])
model.save("charRNN.h5")

Epoch 1/10
31368/31368 [==============================] - 419s 13ms/step - loss: 1.6217
Epoch 2/10
31368/31368 [==============================] - 412s 13ms/step - loss: 1.5343
Epoch 3/10
31368/31368 [==============================] - 412s 13ms/step - loss: 1.5122
Epoch 4/10
31368/31368 [==============================] - 415s 13ms/step - loss: 1.5009
Epoch 5/10
31368/31368 [==============================] - 407s 13ms/step - loss: 1.4933
Epoch 6/10
31368/31368 [==============================] - 409s 13ms/step - loss: 1.4884
Epoch 7/10
31368/31368 [==============================] - 414s 13ms/step - loss: 1.4849
Epoch 8/10
31368/31368 [==============================] - 413s 13ms/step - loss: 1.4820
Epoch 9/10
31368/31368 [==============================] - 416s 13ms/step - loss: 1.4797
Epoch 10/10
31368/31368 [==============================] - 413s 13ms/step - loss: 1.4776


p279 保存模型的两种方法

In [6]:
# 首先加载保存好的模型
model = keras.models.load_model("charRNN.h5")

使用CharRNN模型预测下一个字符，在提供文本前首先要对它进行预处理

In [7]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [8]:
X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis=-1)
# 1st sentence, last char
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

生成假莎士比亚文本

提供一些文本，让模型预测最可能的下一个字母，把它添加在文本末尾。然后以此循环，实际上效果不好：会反复出现相同的单词

使用tf.random.categorical()函数估计的概率，随机选择下一个字符

给定类对数概率/温度，函数会对随机类索引进行采样，接近0的温度倾向于高概率字符，而非常高的温度会给予所有字符相同的概率

In [9]:
# 生成要添加到文本的下一个字符
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [25]:
next_char("How are yo", temperature=1)

'u'

In [10]:
# 反复调用next_char()并添加到文本中
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [11]:
complete_text("t", temperature=0.2)

'the suitors to her father comes and with such serva'

可以添加更多的GRU层和每层有更多的神经元，训练更长时间并添加正则化

当前模型无法学习比n_step更长的模式

LSTM和GRU无法处理很长的序列，序列长度为100是一个临界值

## 有状态的RNN

无状态RNN是每次训练迭代模型都从一个充满零的隐藏状态开始，<font color='aqua'>每个时间步长更新该状态，在最后一个时间步长之后将其丢弃</font>

___如果让RNN在处理一个训练批次后保留这个最终状态并将其用作下一个训练批次的初始状态___ 这样，尽管反向传播只是通过短序列，模型仍可以学习长期模式，这就是有状态RNN

只有当批次中的每个输入序列均从上一个批次中对应序列中断的确切位置开始时，有状态RNN才有意义，因此必须使用顺序和非重合的输入序列，而不能使用混洗的重合队列

因此接下来使用包含单个窗口的批处理

In [8]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [9]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [10]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [11]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [12]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])
model.save("stateful_RNN.h5")

Epoch 1/50
313/313 [==============================] - 12s 18ms/step - loss: 2.6234
Epoch 2/50
313/313 [==============================] - 4s 13ms/step - loss: 2.2466
Epoch 3/50
313/313 [==============================] - 4s 13ms/step - loss: 2.1155
Epoch 4/50
313/313 [==============================] - 4s 13ms/step - loss: 2.0366
Epoch 5/50
313/313 [==============================] - 4s 13ms/step - loss: 1.9855
Epoch 6/50
313/313 [==============================] - 4s 13ms/step - loss: 1.9458
Epoch 7/50
313/313 [==============================] - 4s 13ms/step - loss: 1.9200
Epoch 8/50
313/313 [==============================] - 4s 13ms/step - loss: 1.8957
Epoch 9/50
313/313 [==============================] - 4s 13ms/step - loss: 1.8790
Epoch 10/50
313/313 [==============================] - 4s 13ms/step - loss: 1.8618
Epoch 11/50
313/313 [==============================] - 4s 13ms/step - loss: 1.8486
Epoch 12/50
313/313 [==============================] - 4s 13ms/step - loss: 1.8378
Epoch 13/50


In [13]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [14]:
model = keras.models.load_model("stateful_RNN.h5")
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model

In [15]:
print(complete_text("t"))

NameError: name 'complete_text' is not defined